In [1]:
#!pip install ipynb --upgrade

In [2]:
#!pip install --upgrade pip

In [3]:
#!pip install keras-tcn
#!pip install tensorflow
#!pip install ipynb
#!pip install plotly

In [4]:
import pickle
import time
import os
import yaml
import sys
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
sys.path.append('C:/Users/Alex-/Desktop/Beslutsstödssystem/models-decision-system')
os.chdir("..")
import Decision
import Models

os.chdir("MockPipeline")

os.chdir("../../pipeline")
import ipynb.fs.full.processing as processing
import ipynb.fs.full.features as features
import ipynb.fs.full.training as training
import ipynb.fs.full.analysis as analysis
import ipynb.fs.full.storage as storage
import ipynb.fs.full.visualize as visualize
os.chdir("../models-decision-system/MockPipeline")

In [5]:
import os

In [6]:
def load_yaml(path):
    with open(path, mode='r') as file:
        return yaml.load(file, Loader=yaml.FullLoader)

In [7]:
settings = load_yaml('mock_settings.yaml')

In [8]:
dataframe = processing.create_dataframe(settings)

dataset = features.add(dataframe, settings['features'])

dataset

data_linreg = features.split(dataset, 'linreg', settings['ensemble']['models'][0]['linreg'])

linreg = Models.train_model(data_linreg, 'linreg', settings['ensemble']['models'][0])

data_lstm = features.split(dataset, 'lstm', settings['ensemble']['models'][1]['lstm'])

settings['ensemble']['models'][1]['lstm']['layers']

lstm = Models.train_model(data_lstm, 'lstm', settings['ensemble']['models'][1]['lstm'])

from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import TimeSeriesSplit
import numpy as np
from sklearn import preprocessing
from sklearn import utils

In [9]:
df = pd.read_csv("../MockData.csv")
df_dec = pd.read_csv("../decision.csv")

In [10]:
df_dec = df_dec.set_index("DateTime")
df = df.set_index("DateTime")

In [11]:
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import train_test_split

In [12]:
df

,Close,RoC,S%D,PSY12,ASY5
DateTime,,,,,
2010-01-27,0.817467,0.279074,0.239967,0.272727,0.452398
2010-01-28,0.802634,0.256945,0.173515,0.272727,0.376690
2010-01-29,0.779823,0.260848,0.136936,0.272727,0.291014
2010-01-31,0.782677,0.276472,0.108876,0.363636,0.304610
2010-02-01,0.793195,0.281944,0.078999,0.363636,0.385942
...,...,...,...,...,...
2019-12-26,0.159700,0.475611,0.395299,0.727273,0.525453
2019-12-27,0.175454,0.565530,0.421044,0.818182,0.586624
2019-12-29,0.177701,0.567630,0.451166,0.818182,0.585169


In [13]:
df_dec

,Forecast,Actual,f_momentum,a_momentum,Invest next day
DateTime,,,,,
2018-05-22,0.330828,0.296319,NaN,NaN,Hold
2018-05-23,0.333239,0.299195,NaN,NaN,Hold
2018-05-24,0.328026,0.282542,-0.002801,-0.013777,Sell
2018-05-25,0.322993,0.291667,-0.010246,-0.007529,Hold
2018-05-27,0.310946,0.277665,-0.017081,-0.004877,Buy
...,...,...,...,...,...
2019-12-24,0.168590,0.158172,-0.001180,0.000607,Hold
2019-12-25,0.167789,0.159700,-0.000904,0.003124,Hold
2019-12-26,0.166317,0.175454,-0.002273,0.017282,Hold


In [14]:
col = "Invest next day"
conditions = [df_dec[col] == "Buy"]
choices = ["1"]
df_dec["buy"] = np.select(conditions, choices, default="0")
conditions = [df_dec[col] == "Sell"]
choices = ["1"]
df_dec["sell"] = np.select(conditions, choices, default="0")
conditions = [df_dec[col] == "Hold"]
choices = ["1"]
df_dec["hold"] = np.select(conditions, choices, default="0")

In [15]:
df_dec

,Forecast,Actual,f_momentum,a_momentum,Invest next day,buy,sell,hold
DateTime,,,,,,,,
2018-05-22,0.330828,0.296319,NaN,NaN,Hold,0,0,1
2018-05-23,0.333239,0.299195,NaN,NaN,Hold,0,0,1
2018-05-24,0.328026,0.282542,-0.002801,-0.013777,Sell,0,1,0
2018-05-25,0.322993,0.291667,-0.010246,-0.007529,Hold,0,0,1
2018-05-27,0.310946,0.277665,-0.017081,-0.004877,Buy,1,0,0
...,...,...,...,...,...,...,...,...
2019-12-24,0.168590,0.158172,-0.001180,0.000607,Hold,0,0,1
2019-12-25,0.167789,0.159700,-0.000904,0.003124,Hold,0,0,1
2019-12-26,0.166317,0.175454,-0.002273,0.017282,Hold,0,0,1


In [16]:
df["buy"] = df_dec["buy"]
df["sell"] = df_dec["sell"]
df["hold"] = df_dec["hold"]

In [17]:
df = df.dropna()
df

,Close,RoC,S%D,PSY12,ASY5,buy,sell,hold
DateTime,,,,,,,,
2018-05-22,0.314006,0.480472,0.194738,0.363636,0.479935,0,0,1
2018-05-23,0.296319,0.444085,0.198627,0.363636,0.432865,0,0,1
2018-05-24,0.299195,0.411007,0.185304,0.363636,0.462282,0,1,0
2018-05-25,0.282542,0.349541,0.151715,0.272727,0.400889,0,0,1
2018-05-27,0.291667,0.366831,0.125065,0.272727,0.414903,1,0,0
...,...,...,...,...,...,...,...,...
2019-12-24,0.156576,0.526441,0.519878,0.727273,0.477280,0,0,1
2019-12-25,0.158172,0.504520,0.453905,0.727273,0.476746,0,0,1
2019-12-26,0.159700,0.475611,0.395299,0.727273,0.525453,0,0,1


In [18]:
df.columns[:5]

Index(['Close', 'RoC', 'S%D', 'PSY12', 'ASY5'], dtype='object')

In [19]:
df

,Close,RoC,S%D,PSY12,ASY5,buy,sell,hold
DateTime,,,,,,,,
2018-05-22,0.314006,0.480472,0.194738,0.363636,0.479935,0,0,1
2018-05-23,0.296319,0.444085,0.198627,0.363636,0.432865,0,0,1
2018-05-24,0.299195,0.411007,0.185304,0.363636,0.462282,0,1,0
2018-05-25,0.282542,0.349541,0.151715,0.272727,0.400889,0,0,1
2018-05-27,0.291667,0.366831,0.125065,0.272727,0.414903,1,0,0
...,...,...,...,...,...,...,...,...
2019-12-24,0.156576,0.526441,0.519878,0.727273,0.477280,0,0,1
2019-12-25,0.158172,0.504520,0.453905,0.727273,0.476746,0,0,1
2019-12-26,0.159700,0.475611,0.395299,0.727273,0.525453,0,0,1


In [20]:
def foofoofoo(lblz):
    container = []
    for row in lblz:
        word = ''
        for value in row:
            word = word + str(value)
        container.append(word)
    return container

In [21]:
labels = df[["buy", "sell", "hold"]].to_numpy()

In [22]:
labels = foofoofoo(labels)

In [23]:
forecast_days = 1
#df['Label'] = df['Close'].shift(-forecast_days)
df_dec.dropna()
data = df
data = data.dropna(how="any")
x = data[df.columns[:5]].to_numpy()
y = data[df.columns[-4]].to_numpy()
y2 = data[df.columns[-3:]].to_numpy()

## Dataset y


In [24]:
def create_split(x, y):
    tss = TimeSeriesSplit(n_splits=5)
    for train_index, test_index in tss.split(x):
        x_train, x_test = x[train_index], x[test_index]
        y_train, y_test = y[train_index], y[test_index]
    return x_train, y_train, x_test, y_test

In [25]:
x_train, y_train, x_test, y_test = create_split(x, y)
x_train_2, y_train_2, x_test_2, y_test_2 = create_split(x, y2)

In [26]:
y_test_2 = foofoofoo(y_test_2)
y_train_2 = foofoofoo(y_train_2)

## Dataset y2

In [27]:
#from sklearn.multiclass import OneVsRestClassifier
#from sklearn.svm import LinearSVC
#OneVsRestClassifier(LinearSVC(random_state=0)).fit(x, y).predict(x)

In [28]:
from sklearn.preprocessing import MinMaxScaler

In [29]:
scaler = MinMaxScaler(feature_range=(0,1))

x_train = scaler.fit_transform(x_train)
x_test= scaler.transform(x_test)

In [30]:
x = scaler.transform(x)

lab_enc = preprocessing.LabelEncoder()
encoded = lab_enc.fit_transform(y_train)
print(training_scores_encoded)
print(utils.multiclass.type_of_target(y_train))
print(utils.multiclass.type_of_target(y_train.astype('int')))
print(utils.multiclass.type_of_target(training_scores_encoded))

In [31]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from pprint import pprint
from sklearn import metrics

# Gridsearch random forest

https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74


### Functions

In [32]:
def random_search(settings):
    
    # CREATE THE CORRECT MODEL TYPE
    model = select_model(settings)

    # CREATE THE RANDOMIZED SEARCH CV
    rscv = RandomizedSearchCV(estimator=model,
                                  param_distributions=settings['random_params'],
                                  n_iter=100,
                                  cv = 3,
                                  verbose=2,
                                  random_state=42,
                                  n_jobs=-1)

    # FIT THE RANDOM SEARCH
    rscv.fit(settings["x"], settings["y"])
    
    # SELECT THE BEST ESTIMATOR
    best_random = rscv.best_estimator_

    
    
    return rscvm, rscv.best_params_

In [33]:
def grid_search(model, dataset, settings):

    
    # SELECT MODEL
    model = select_model(settings)

    # CREATE GRID SEARCH CV
    grid_search = GridSearchCV(estimator = model, param_grid = settings["grid_params"], 
                          cv = settings["cv"], n_jobs = settings["n_jobs"], verbose = 2)
    grid_search.fit(dataset["x"], dataset["y"])
    best_grid = grid_search.best_estimator_

    return best_grid

In [34]:
def select_model(settings):
    """Select a classification model type. 
    Supported models:
    Random Forest Classifier (default)"""
    if(settings["model"] == "rfc"):
        model = RandomForestClassifier()
    else:
        model = RandomForestClassifier()
    return model

In [52]:
from sklearn import preprocessing

In [61]:
def grid_search_model(model, dataset, settings):
    """Runs a grid search on the given model to define the best parameters for the model. 
    
    Parameters:
    -----------
    model: sklearn model 
    unfitted model
    dataset: dict
    Dictionary that contains x_train, y_train, features and labels
    
    Returns:
    --------
    model: sklearn model
    Same type of model as the input model, with optimized parameters
    """
    # FIT GRIDSEARCH
    if (settings["scoring"] == "roc_auc"):
        grid_search = GridSearchCV(estimator = model, param_grid = settings["grid_params"], 
                          cv = settings["cv"], n_jobs = settings["n_jobs"], verbose = 2, scoring = settings["scoring"])

        y = preprocessing.label_binarize(dataset["labels"], classes=[0, 1, 2])
        grid_search.fit(dataset["features"], y)

        # PICK OUT THE BEST MODEL FROM GRID SEARCH
        model = grid_search.best_estimator_

        # RUN MODEL ON BEST PARAMS   
        model.fit(dataset["x_train"], dataset["y_train"])

        # RETURN MODEL WITH BEST PARAMS    
        return model
    else:
        grid_search = GridSearchCV(estimator = model, param_grid = settings["grid_params"], 
                          cv = settings["cv"], n_jobs = settings["n_jobs"], verbose = 2)


        grid_search.fit(dataset["features"], dataset["labels"])

        # PICK OUT THE BEST MODEL FROM GRID SEARCH
        model = grid_search.best_estimator_

        # RUN MODEL ON BEST PARAMS   
        model.fit(dataset["x_train"], dataset["y_train"])

        # RETURN MODEL WITH BEST PARAMS    
        return model    

In [62]:
from sklearn import preprocessing
def evaluate(model, test_features, test_labels, encode=True):
    """Evaluate a classifcation model, for example Random Forest Classification
    
    Parameters:
    model: sklearn classification model
    test_features: np.array of features
    test_labels: np.array of labels 
    encode (default=True): boolean, encodes values to int if set to true
    
    Returns:
    accuracy: float, metrics.accuracy_score
    precision: float, metrics.precision_score
    recall: float, metrics.precision_score
    confusion: 
    """
    # PREDICT
    
    # PULL ACCURACY FROM METRICS
    try:
        predict = model.predict(test_features)
        class_report = metrics.classification_report(test_labels, predict)
    
        predict_proba = model.predict_proba(test_features)
        roc_auc = metrics.roc_auc_score(test_labels, predict_proba, multi_class='ovo', average='macro')
        
        print(class_report)
        print(roc_auc)
        return "Double"
    except:
        predict = model.predict(test_features)
        class_report = metrics.classification_report(test_labels, predict)
        
        print(class_report)
        return "Single"

In [37]:
# C:\Users\Alex-\Desktop\Beslutsstödssystem\models-decision-system\MockPipeline
import pickle
def load_pickle(path):
    return pickle.load(open(path, 'rb'))

In [38]:
%matplotlib notebook
#new_data = load_pickle("cls-dataset.pickle")
x_test = pickle.load(open("test-dataset.pickle", 'rb'))

In [39]:
y_test = pd.read_csv("sick-dataset.csv")

In [40]:
y_test = y_test["label"]

In [41]:
train_data = pd.read_csv("train-dataset.csv")

In [42]:
x_train = train_data[train_data.columns[1:4]].to_numpy()

In [43]:
y_train = train_data[train_data.columns[4]].to_numpy()

# Random search grid

In [44]:
x = np.concatenate((x_train, x_test))
y = np.concatenate((y_train, y_test))

In [45]:
dataset = {"x_train": x_train, 
           "y_train": y_train,
           "x_test": x_test,
           "y_test": y_test,
           "features": x, # Full data used in grid searching
           "labels": y # Full data used in grid searching
          }

In [63]:
def run_model(model, data, grid_settings):
    best_model = grid_search_model(model, data, grid_settings)
    print(best_model)
    best_model.fit(data["x_train"], data["y_train"])
    results = evaluate(best_model, data["x_test"], data["y_test"])
    #print(results)
    #print(roc_auc)

## Random forest

In [64]:
grid_settings = {
    "cv": 3, # number of folds
    "n_jobs": -1, # processors to use during grid search
    "scoring": "roc_auc",
    'grid_params': { # parameters for grid search (specific to each model)
        'n_estimators': [10, 50, 100, 500], # int, default=100
        'max_features': ['log2', 'sqrt'], # {“sqrt”, “log2”}, int or float, default=”auto”
        'max_depth': [10,100, None], # int, default=None
        'min_samples_split': [3, 10], # int or float, default=2
        'min_samples_leaf': [2, 4], # int or float, default=1
        'bootstrap': [True, False], # bool, default=True
        "class_weight": ["balanced"], # {“balanced”, “balanced_subsample”}, dict or list of dicts, default=None
    }
}
rfc = RandomForestClassifier()
run_model(rfc, dataset, grid_settings)

Fitting 3 folds for each of 192 candidates, totalling 576 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 186 tasks      | elapsed:   14.1s
[Parallel(n_jobs=-1)]: Done 389 tasks      | elapsed:   30.9s


RandomForestClassifier(class_weight='balanced', max_depth=100,
                       max_features='sqrt', min_samples_leaf=2,
                       min_samples_split=10, n_estimators=10)
              precision    recall  f1-score   support

           0       0.19      0.48      0.27       119
           1       0.14      0.13      0.14       114
           2       0.57      0.30      0.39       385

    accuracy                           0.30       618
   macro avg       0.30      0.30      0.27       618
weighted avg       0.41      0.30      0.32       618

0.4915283976429487


[Parallel(n_jobs=-1)]: Done 576 out of 576 | elapsed:   51.9s finished


## Logistical Regression

In [70]:
grid_settings = {
    "cv": 3, # number of folds
    "n_jobs": -1, # processors to use during grid search
    "scoring": "nope",
    'grid_params': {
        "C": [0.1, 1, 10, 100],
        "penalty": ["l2",  "none"], # ["l1", "l2", "elasticnet", "none"]
        "solver": ["newton-cg", "lbfgs", "sag", "saga"], # ["newton-cg", "lbfgs", "liblinear", "sag", "saga"], default=’lbfgs’
        "multi_class": ["auto", "multinomial", "ovo", "ovr"], # [‘auto’, ‘ovr’, ‘multinomial’], default=’auto’
        "max_iter": [1000],
        "random_state": [0],
        "class_weight": ["balanced"]
    }
}
logreg = LogisticRegression()
run_model(logreg, dataset, grid_settings)

Fitting 3 folds for each of 128 candidates, totalling 384 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 361 out of 384 | elapsed:    1.3s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done 384 out of 384 | elapsed:    1.4s finished
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1320: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1320: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1320: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(


LogisticRegression(C=0.1, class_weight='balanced', max_iter=1000,
                   multi_class='ovr', penalty='none', random_state=0,
                   solver='newton-cg')
              precision    recall  f1-score   support

           0       0.19      0.60      0.28       119
           1       0.21      0.27      0.24       114
           2       0.54      0.13      0.21       385

    accuracy                           0.24       618
   macro avg       0.31      0.33      0.24       618
weighted avg       0.41      0.24      0.23       618

0.47077998663580267


## SVC

In [ ]:
grid_settings = {
    "cv": 3, # number of folds
    "n_jobs": -1, # processors to use during grid search
    "grid_params":{
        "C": [0.1, 1,10,100],
        "kernel": ["linear", "rbf"],
        "degree": [1,3], #int, default=3
        #"gamma": [],
        "shrinking": [True, False],
        "class_weight": ["balanced"],
        #"decision_function_shape": ["ovo", "ovr"]
    }
}
model = SVC(probability=True)
run_model(model, dataset, grid_settings)

## GaussianNB

In [ ]:
from sklearn.naive_bayes import GaussianNB
grid_settings = {
    "cv": 3, # number of folds
    "n_jobs": -1, # processors to use during grid search
    "grid_params":{
        "var_smoothing": [1e-9, 1e-10, 1e-5]
    }
}
model = GaussianNB()
run_model(model , dataset, grid_settings)

# Complemntary NB

In [ ]:
from sklearn.naive_bayes import ComplementNB
grid_settings = {
    "cv": 3, # number of folds
    "n_jobs": -1, # processors to use during grid search
    "grid_params":{

        "alpha": [0.5, 1],
        "norm": [True, False],
        
    }
}
model = ComplementNB()
run_model(model , dataset, grid_settings)

# Ridge Classifier

In [ ]:
from sklearn.linear_model import RidgeClassifier
grid_settings = {
    "cv": 3, # number of folds
    "n_jobs": -1, # processors to use during grid search
    "grid_params":{
        "class_weight": ["balanced"],
        "alpha": [0.5, 1],        
    }
}
model = RidgeClassifier()
run_model(model , dataset, grid_settings)

# Linear Discriminant Analysis

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
grid_settings = {
    "cv": 3, # number of folds
    "n_jobs": -1, # processors to use during grid search
    "grid_params":{
        "solver": ["svd", "lsqr", "eigen"],
        "shrinkage": [0.25, 0.50, 0.75, 1]
    }
}
model = LinearDiscriminantAnalysis()
run_model(model , dataset, grid_settings)

# Quadratic Discriminant Analysis

In [ ]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
grid_settings = {
    "cv": 3, # number of folds
    "n_jobs": -1, # processors to use during grid search
    "grid_params":{
        "reg_param": [0, 0.25, 0.5, 0.75],
        "tol": [0.25, 0.50, 0.75, 1]
    }
}
model = QuadraticDiscriminantAnalysis()
run_model(model , dataset, grid_settings)

# SGD Classifier

Very varied results

In [ ]:
from sklearn.linear_model import SGDClassifier
grid_settings = {
    "cv": 3, # number of folds
    "n_jobs": -1, # processors to use during grid search
    "grid_params":{
        "epsilon": [0.18],
        "penalty": ["l1"],
        "class_weight": ["balanced", None],
        "early_stopping": [True, False],
        "eta0": [0.1,0.5,1],
        "learning_rate": ["constant", "optimal", "invscaling", "adaptive"]
    }
}
model = SGDClassifier()
run_model(model , dataset, grid_settings)

SGDClassifier(class_weight='balanced', early_stopping=True, epsilon=0.18,
              eta0=0.1, learning_rate='invscaling')
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       119
           1       0.22      0.07      0.11       114
           2       0.63      0.95      0.76       385

    accuracy                           0.61       618
   macro avg       0.28      0.34      0.29       618
weighted avg       0.43      0.61      0.49       618

SGDClassifier(class_weight='balanced', early_stopping=True, epsilon=0.18,
              eta0=0.1, learning_rate='invscaling', penalty='l1')
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       119
           1       0.19      0.82      0.31       114
           2       0.73      0.24      0.36       385

    accuracy                           0.30       618
   macro avg       0.31      0.36      0.22       618
weighted avg       0.49      0.30      0.28       618

# Decision Tree Classifier

Splitter = random changes results alot from each run

In [ ]:
from sklearn.tree import DecisionTreeClassifier
grid_settings = {
    "cv": 3, # number of folds
    "n_jobs": -1, # processors to use during grid search
    "grid_params":{
        "criterion": ["gini", "entropy"],
        "splitter": ["best"],
        "min_samples_split": [1,2],
        "min_samples_leaf": [1,2],
        "max_features": ["auto", "aqrt", "log2", None],
        "class_weight": ["balanced"]
    }
}
model = DecisionTreeClassifier()
run_model(model , dataset, grid_settings)

## 3-way confusion matrix

| | pred buy | pred sell | pred hold |
| --------- |:---------:|:---------:|----------:|        
| true buy  | | | |
| true sell | | | |
| true hold | | | |

| Tables        | Are           | Cool  |
| ------------- |:-------------:| -----:|
| col 3 is      | right-aligned | $1600 |
| col 2 is      | centered      |   $12 |
| zebra stripes | are neat      |    $1 |